In [5]:
#Look for tickers with upcoming earnings(max days in advance parameter)

#Look at HV/IV ratio, look for absolue IV

#Compare expected move to historical moves
  #Expected move, find nearest expiry compute 1D straddle
    #1 day straddle calculation look at days and hours (if current time is within trading day)

#Have option to add constraints
  #HV/IV
  #Absolute IV
  #Liquidity constraints (B-A/Premium)
  #Premium constraints
  #Earnings performance of similar stocks

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, time, timedelta
import requests

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay

import yfinanceHelper as yfh
import FinanceTools as ft
import FinanceScrapers as fs
import CalendarTools as ct
import OptionsEarningsTools as oet

In [2]:
oet.getEarningsScreen('GME', earningsTime='PM', displayEarningsTable=False)

/Users/grantlau/Documents/Fall2023/APS502/PythonFiles/.venv/lib/python3.9/site-packages/yfinance/base.py:2093: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(data)[0]


ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

In [ ]:
def getSigmaForward(sigma_1, sigma_2, t_1, t_2):
  return np.sqrt((sigma_2**2*t_2-sigma_1**2*t_1)/(t_2-t_1))

def getExpectedDailyMove(yfTicker, nextEarningsDate, startDate = datetime.today()):
  #https://www.trading-volatility.com/Trading-Volatility.pdf - Chapter 6.4
  #Note T is in trading days not years here
  expiries = tuple(datetime.strptime(date_str, "%Y-%m-%d") for date_str in yfTicker.options)
  expiries = tuple(datetime.combine(expiry.date(), time(16,0)) for expiry in expiries)
  expiryIndex_T1 = min(range(len(expiries)), key=lambda i: (expiries[i] < nextEarningsDate, abs(expiries[i] - nextEarningsDate)))
  expiry_T1 = expiries[expiryIndex_T1]

  callChain_T1, putChain_T1, underlyingInfo_T1 = yfh.getYfOptionChain(yfTicker, expiry_T1)
  sigma_T1 = yfh.getYfImpliedVol(callChain_T1, putChain_T1, underlyingInfo_T1, underlyingInfo_T1['regularMarketPrice'])
  T1 = ct.tradingDaysToExpiry(expiryDate = expiry_T1, startDate = startDate) #TODO calculate time in days between today and expiry_jump

  if expiryIndex_T1 > 0: #expiry exists before earnings, use T0 to calculate sigma_diffusive
    expiryIndex_T0 = expiryIndex_T1 - 1
    expiry_T0 = expiries[expiryIndex_T0]
    callChain_T0, putChain_T0, underlyingInfo_T0 = yfh.getYfOptionChain(yfTicker, expiry_T0)
    sigma_diffusive = yfh.getYfImpliedVol(callChain_T0, putChain_T0, underlyingInfo_T0, underlyingInfo_T0['regularMarketPrice'])
  else: #Earnings expiry is closest expiry, use T2 and forward implied vol to calculate sigma_diffusive
    expiryIndex_T2 = expiryIndex_T1+1
    expiry_T2 = expiries[expiryIndex_T2]
    callChain_T2, putChain_T2, underlyingInfo_T2 = yfh.getYfOptionChain(yfTicker, expiry_T2)
    sigma_T2 = yfh.getYfImpliedVol(callChain_T2, putChain_T2, underlyingInfo_T2, underlyingInfo_T2['regularMarketPrice'])
    T2 = ct.tradingDaysToExpiry(expiryDate = expiry_T2, startDate = startDate)
    sigma_diffusive = getSigmaForward(sigma_T1, sigma_T2, T1, T2)
  sigma_jump = np.sqrt(sigma_T1**2*T1-sigma_diffusive**2*(T1-1))

  expectedDailyMove = sigma_jump/np.sqrt(252)
  #TODO normalize for (subtract) index term structure
  return expectedDailyMove

In [ ]:
def getEarningsScreen(ticker, earningsTime = 'PM', displayEarningsTable = False):
  yfTicker = yfh.getYfTicker(ticker)
  earningsTable = getEarningsTable(yfTicker, earningsTime = earningsTime)
  avgAbs1DMove = earningsTable['1D Move'].abs().mean()
  positive1DMoves = (earningsTable['1D Move']>0).sum()
  reportedQuarters = earningsTable['Reported EPS'].count()
  today = datetime.today()
  nextEarningsIndex = min(range(len(earningsTable.index)), key=lambda i: (earningsTable.index[i] < today, abs(earningsTable.index[i] - today)))
  nextEarningsDate = earningsTable.index[nextEarningsIndex]
  implied1DMove = getExpectedDailyMove(yfTicker, nextEarningsDate)
  data = {
      'Symbol': [yfTicker.info['symbol']],
      'CurrentPrice': [yfTicker.info['currentPrice']],
      'EarningsDate': [nextEarningsDate.date()],
      'EarningsTime': [earningsTime],
      'AvgAbs1DMove': [avgAbs1DMove],
      'Implied1DMove': [implied1DMove],
      'Positive1DMoves': [positive1DMoves],
      'ReportedQuarters': [reportedQuarters]
  }
  earningsScreenDF = pd.DataFrame(data)
  if displayEarningsTable:
    display(earningsTable)
  return earningsScreenDF

def getEarningsTable(yfTicker, earningsTime = 'PM'):
  yfEarningsDates = yfTicker.earnings_dates
  #Get at least 8 previous quarters of earnings if possible
  if yfEarningsDates['Reported EPS'].count()<8:
    rowsNeeded = len(yfEarningsDates)+8-yfEarningsDates['Reported EPS'].count()
    yfEarningsDates = yfTicker.get_earnings_dates(limit = rowsNeeded)

  yfEarningsDates = yfEarningsDates.tz_localize(None)
  if earningsTime == 'AM':
    yfEarningsDates.index = yfEarningsDates.index.map(lambda x: x.replace(hour=9, minute=0, second=0, microsecond=0))
  else:
    yfEarningsDates.index = yfEarningsDates.index.map(lambda x: x.replace(hour=17, minute=0, second=0, microsecond=0))
  firstEarningsDate = yfEarningsDates.index[-1]
  yfHistory = yfh.getYfHistory(yfTicker, ct.getPreviousTradingDay(firstEarningsDate))
  spx_yfHistory = yfh.getYfHistory(yfh.getYfTicker('^SPX'), ct.getPreviousTradingDay(firstEarningsDate))
  if earningsTime == "AM":
    subsequentLogReturns = ft.getLogReturns(yfHistory['Close']).shift(0)
    spx_subsequentLogReturns = ft.getLogReturns(spx_yfHistory['Close']).shift(0)
  else: #earningsTime == 'PM'
    subsequentLogReturns = ft.getLogReturns(yfHistory['Close']).shift(-1) #-1
    spx_subsequentLogReturns = ft.getLogReturns(spx_yfHistory['Close']).shift(-1)
  oneDayMoves = pd.merge(yfEarningsDates, subsequentLogReturns, left_on=yfEarningsDates.index.date, right_on=subsequentLogReturns.index.date, how = 'left')['Close']
  spx_oneDayMoves = pd.merge(yfEarningsDates, spx_subsequentLogReturns, left_on=yfEarningsDates.index.date, right_on=spx_subsequentLogReturns.index.date, how = 'left')['Close']
  yfEarningsDates['1D Move'] = oneDayMoves.values
  yfEarningsDates['SPX 1D Move'] = spx_oneDayMoves.values
  return yfEarningsDates

def getUpcomingEarnings(startDate, endDate, minMarketCap = None, universe = None):
  earningsCal = fs.scrapeEarningsCalendar(startDate, endDate, minMarketCap)
  if universe != None:
    indexDF = fs.scrapeIndex(universe)
    earningsCal = pd.merge(earningsCal, indexDF, left_on='symbol', right_on='Ticker', how='inner')
  earningsScreens = []
  for index, row in earningsCal.iterrows():
    earningsTime = "AM" if row['time'] == 'time-pre-market' else "PM"
    earningsScreens.append(getEarningsScreen(row['symbol'], earningsTime))
  upcomingEarnings = pd.concat(earningsScreens, ignore_index=True)
  return upcomingEarnings

In [ ]:
getEarningsScreen('GME', earningsTime='PM', displayEarningsTable=True)

EPS Estimate  Reported EPS  Surprise(%)   1D Move  \
Earnings Date                                                            
2024-12-04 17:00:00           NaN           NaN          NaN       NaN   
2024-09-04 17:00:00           NaN           NaN          NaN       NaN   
2024-06-05 17:00:00           NaN           NaN          NaN       NaN   
2024-03-19 17:00:00          0.28           NaN          NaN       NaN   
2024-03-19 17:00:00          0.28           NaN          NaN       NaN   
2023-12-06 17:00:00         -0.09           NaN       1.0000  0.097513   
2023-12-05 17:00:00         -0.08           NaN       1.0000 -0.004706   
2023-09-06 17:00:00         -0.14         -0.03       0.7877  0.007439   
2023-06-07 17:00:00         -0.12         -0.14      -0.1667 -0.197060   
2023-03-21 17:00:00         -0.13          0.16       2.2030  0.301887   
2022-12-07 17:00:00         -0.28         -0.31      -0.1205  0.107649   
2022-09-07 17:00:00         -0.38         -0.35       0.0728  0.071817   
2022-06-01 17:00:00         -0.36         -0.52      -0.4345  0.098749   
2022-03-17 17:00:00          0.21         -0.46      -3.2208  0.034627   
2021-12-08 17:00:00         -0.13         -0.35      -1.6861 -0.108725   

                     SPX 1D Move  
Earnings Date                     
2024-12-04 17:00:00          NaN  
2024-09-04 17:00:00          NaN  
2024-06-05 17:00:00          NaN  
2024-03-19 17:00:00          NaN  
2024-03-19 17:00:00          NaN  
2023-12-06 17:00:00     0.007937  
2023-12-05 17:00:00    -0.003914  
2023-09-06 17:00:00    -0.003216  
2023-06-07 17:00:00     0.006170  
2023-03-21 17:00:00    -0.016600  
2022-12-07 17:00:00     0.007494  
2022-09-07 17:00:00     0.006589  
2022-06-01 17:00:00     0.018263  
2022-03-17 17:00:00     0.011595  
2021-12-08 17:00:00    -0.007207

Symbol  CurrentPrice EarningsDate EarningsTime  AvgAbs1DMove  Implied1DMove  \
0    GME         18.12   2024-03-19           PM      0.103017       0.152346   

   Positive1DMoves  ReportedQuarters  
0                7                 8

In [5]:
startDate = datetime.today()
endDate = datetime.today()+timedelta(days=30)
minMarketCap = 100000000000
earningsCalendar = fs.scrapeEarningsCalendar(startDate, endDate, minMarketCap)

In [6]:
earningsCalendar

lastYearRptDt  lastYearEPS               time symbol  \
0      1/12/2023         1.82  time-not-supplied    TSM   
1      1/13/2023         3.57    time-pre-market    JPM   
2      1/13/2023         5.34    time-pre-market    UNH   
3      1/13/2023         0.85    time-pre-market    BAC   
4      1/13/2023         0.67    time-pre-market    WFC   
..           ...          ...                ...    ...   
60     2/02/2023         1.87  time-not-supplied   SONY   
61     2/02/2023         0.75  time-not-supplied   SBUX   
62     2/02/2023         0.06  time-not-supplied   MUFG   
63     2/02/2023         1.67  time-not-supplied   GILD   
64     2/02/2023         1.82    time-pre-market    BMY   

                                               name     marketCap  \
0   Taiwan Semiconductor Manufacturing Company Ltd.  5.393519e+11   
1                             J P Morgan Chase & Co  4.917605e+11   
2                   UnitedHealth Group Incorporated  4.869454e+11   
3                       Bank of America Corporation  2.664554e+11   
4                             Wells Fargo & Company  1.787493e+11   
..                                              ...           ...   
60                           Sony Group Corporation  1.176590e+11   
61                            Starbucks Corporation  1.064747e+11   
62               Mitsubishi UFJ Financial Group Inc  1.054874e+11   
63                            Gilead Sciences, Inc.  1.037205e+11   
64                     Bristol-Myers Squibb Company  1.073538e+11   

   fiscalQuarterEnding  epsForecast  noOfEsts               earningsDate  
0             Dec/2023         1.34       3.0 2024-01-11 01:05:48.829528  
1             Dec/2023         3.64      12.0 2024-01-12 01:05:48.829528  
2             Dec/2023         5.98      11.0 2024-01-12 01:05:48.829528  
3             Dec/2023         0.68      12.0 2024-01-12 01:05:48.829528  
4             Dec/2023         1.16       9.0 2024-01-12 01:05:48.829528  
..                 ...          ...       ...                        ...  
60            Dec/2023         1.65       4.0 2024-02-01 01:05:48.829528  
61            Dec/2023         0.95      12.0 2024-02-01 01:05:48.829528  
62            Dec/2023          NaN       NaN 2024-02-01 01:05:48.829528  
63            Dec/2023         1.78      12.0 2024-02-01 01:05:48.829528  
64            Dec/2023         1.73      10.0 2024-02-02 01:05:48.829528  

[65 rows x 10 columns]

In [7]:
oet.getSigmaForward(0.2, 0.4, 10, 12)

0.8717797887081348

In [ ]:
yfTicker.news

[{'uuid': '1a65cc97-50cd-36fc-95ea-d081f9170c90',
  'title': 'Where Will Walgreens Boots Alliance Be in 1 Year?',
  'publisher': 'Motley Fool',
  'link': 'https://finance.yahoo.com/m/1a65cc97-50cd-36fc-95ea-d081f9170c90/where-will-walgreens-boots.html',
  'providerPublishTime': 1703171700,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/Y3B4Hiv8IWf.ipxAfoOfFQ--~B/aD03MjA7dz0xMDc4O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/bb7badbe92e86a3ccad78c6d2b3cb143',
     'width': 1078,
     'height': 720,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/MO53ikaM9pDdiwL2w49.wQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/bb7badbe92e86a3ccad78c6d2b3cb143',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['WBA']},
 {'uuid': '93dc94ef-1dd8-35dc-8bcd-f3bc859735ad',
  'title': 'Walgreens marketing chief let go amid last m